In [68]:
import pandas as pd  # module for working with data sets (need for working inner classes and modules)
import traceback
import datetime
import subprocess
import warnings
import numpy as np
import os  # module for working with operating system catalog structure
import openpyxl  # module for working with Excel files
import time  # module for working with date and time
import pyodbc  # module for working with databases
import pyxlsb
import threading

from datetime import datetime  # Module for working with date and time data
from tkinter.filedialog import askopenfilename  # Module for open file with win gui

# Ignoring pandas version errors
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))

# Event for new thread
event = threading.Event()

# show an "Open" dialog box and return the path to the selected file
flag = False
filename_comp = askopenfilename(title="Select file for compare", filetypes=[("Excel Files", "*.xlsx"), ("Excel Binary Workbook", "*.xlsb")])
database_root = askopenfilename(title="Select database", filetypes=[('*.mdb', '*.accdb')]).replace('/', '\\')
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    fr'DBQ={database_root};'
    )

table = input("Input table's name : ")
with pyodbc.connect(conn_str) as conn:
    query = f'''SELECT * FROM {table}'''
    new_df = pd.read_sql(query, conn)

start_time = time.time()

#Columns for changing dataframe
convert_columns = ['Дата выпуска РД по договору подрядчика', 
                   'Дата выпуска РД по графику с Заказчиком', 
                   'Дата статуса Заказчика', 
                   'Ожидаемая дата выдачи РД в производство', 
                   'Дата выпуска РД по договору подрядчика_new',
                   'Дата выпуска РД по графику с Заказчиком_new',
                   'Дата статуса Заказчика_new',
                   'Ожидаемая дата выдачи РД в производство_new']

clmns = ['Пакет РД', 'Статус Заказчика', 'Текущая ревизия', 'Статус текущей ревизии', 
           'Дата выпуска РД по договору подрядчика', 'Дата выпуска РД по графику с Заказчиком', 
           'Дата статуса Заказчика', 'Ожидаемая дата выдачи РД в производство']

inf_columns = ['Наименование объекта/комплекта РД',
               'Коды работ по выпуску РД', 
               'Пакет РД', 
               'Код KKS документа',
               'Статус Заказчика', 
               'Текущая ревизия', 
               'Статус текущей ревизии',
               'Дата выпуска РД по договору подрядчика',
               'Дата выпуска РД по графику с Заказчиком',
               'Дата статуса Заказчика',
               'Ожидаемая дата выдачи РД в производство', 
               'Письма',
               'Источник информации', 
               'Разработчики РД (актуальные)', 
               'Статус РД в 1С']

changed_cols = ['Коды работ по выпуску РД',
        'Наименование объекта/комплекта РД',
       'Код KKS документа',
       'Пакет РД',
       'Статус Заказчика', 
       'Текущая ревизия', 
       'Статус текущей ревизии',
       'Дата выпуска РД по договору подрядчика',
       'Дата выпуска РД по графику с Заказчиком', 'Дата статуса Заказчика',
       'Ожидаемая дата выдачи РД в производство',
       'Разработчики РД (актуальные)']

base_columns = ['Система',
                'Наименование объекта/комплекта РД',
                'Коды работ по выпуску РД',
                'Тип',
                'Пакет РД',	'Код KKS документа',
                'Статус Заказчика',
                'Текущая ревизия',
                'Статус текущей ревизии',
                'Дата выпуска РД по договору подрядчика',
                'Дата выпуска РД по графику с Заказчиком',
                'Дата статуса Заказчика',	
                'Ожидаемая дата выдачи РД в производство',	
                'Письма',	
                'Источник информации',	
                'Разработчики РД (актуальные)',	
                'Объект',	
                'WBS',	
                'КС',	
                'Примечания']

# Function for result-file
def new_threading(df_1, df_2):

    def change(df):
        if ~pd.isna(df['Статус текущей ревизии']):
            return df['Статус РД в 1С']
        else:
            return df['Статус текущей ревизии']

    #  Preparation columns list with necessary information
    mdf1_columns = ['Система',
                'Наименование объекта/комплекта РД_new',
                'Коды работ по выпуску РД',
                'Тип',
                'Пакет РД_new',	'Код KKS документа',
                'Статус Заказчика_new',
                'Текущая ревизия_new',
                'Статус текущей ревизии_new',
                'Дата выпуска РД по договору подрядчика_new',
                'Дата выпуска РД по графику с Заказчиком_new',
                'Дата статуса Заказчика_new',	
                'Ожидаемая дата выдачи РД в производство_new',	
                'Письма_new',	
                'Источник информации_new',	
                'Разработчики РД (актуальные)_new',	
                'Объект_new',	
                'WBS_new',	
                'КС',	
                'Примечания',
                'Статус РД в 1С']
    
    mdf2_columns = ['Система',
                'Наименование объекта/комплекта РД',
                'Коды работ по выпуску РД',
                'Тип',
                'Пакет РД_new',	'Код KKS документа_new',
                'Статус Заказчика_new',
                'Текущая ревизия_new',
                'Статус текущей ревизии_new',
                'Дата выпуска РД по договору подрядчика_new',
                'Дата выпуска РД по графику с Заказчиком_new',
                'Дата статуса Заказчика_new',	
                'Ожидаемая дата выдачи РД в производство_new',	
                'Письма_new',	
                'Источник информации_new',	
                'Разработчики РД (актуальные)_new',	
                'Объект_new',	
                'WBS_new',	
                'КС',	
                'Примечания',
                'Статус РД в 1С']
    
    new_columns = ['Система',
                'Наименование объекта/комплекта РД',
                'Коды работ по выпуску РД',
                'Тип',
                'Пакет РД',	'Код KKS документа',
                'Статус Заказчика',
                'Текущая ревизия',
                'Статус текущей ревизии',
                'Дата выпуска РД по договору подрядчика',
                'Дата выпуска РД по графику с Заказчиком',
                'Дата статуса Заказчика',	
                'Ожидаемая дата выдачи РД в производство',	
                'Письма',	
                'Источник информации',	
                'Разработчики РД (актуальные)',	
                'Объект',	
                'WBS',	
                'КС',	
                'Примечания',
                'Статус РД в 1С']

    #  Generate temporary dataframe for next appending for changed documents
    print('Initiate dataframe with changed documents')
    tmp_df = df_1[df_1['_merge'] == 'both'][mdf1_columns]
    tmp_df.columns = new_columns

    
    # Generate result dataframe, with remove document firstly
    print('Generating result dataframe, with remove document firstly')
    changed_df = df_2[df_2['_merge'] == 'left_only'][new_columns]
    tmp_df = pd.concat([changed_df, tmp_df])

    #  Generate result dataframe 
    print('Generating result dataframe ')
    changed_df = df_2[df_2['_merge'] == 'both'][mdf2_columns]
    changed_df.columns = new_columns
    changed_df = pd.concat([changed_df, tmp_df])

    # Changing dataframe
    print('Changing dataframe')
    changed_df['Дата выпуска РД по договору подрядчика'] = pd.to_datetime(changed_df['Дата выпуска РД по договору подрядчика'], dayfirst=True).dt.date
    changed_df['Дата выпуска РД по графику с Заказчиком'] = pd.to_datetime(changed_df['Дата выпуска РД по графику с Заказчиком'], dayfirst=True).dt.date
    changed_df['Дата статуса Заказчика'] = pd.to_datetime(changed_df['Дата статуса Заказчика'], dayfirst=True).dt.date
    changed_df['Ожидаемая дата выдачи РД в производство'] = changed_df['Ожидаемая дата выдачи РД в производство'].apply(lambda row: row if row in ['в производстве', 'В производстве', None]  else pd.to_datetime(row).date())
    changed_df['Объект'] = changed_df['Объект'].apply(lambda row: changed_df['Коды работ по выпуску РД'].str.slice(0, 5) if pd.isna(row) else row)
    changed_df['Статус текущей ревизии'] = changed_df.apply(change, axis = 1)
    changed_df['WBS'] = changed_df['WBS'].apply(lambda row: row if ~pd.isna(row) else changed_df['Коды работ по выпуску РД'].apply(lambda row: row[6 : row.find('.', 6)]))

    event.wait()
    comf_ren = input('Use standard file name (y/n): ')
    while comf_ren not in 'YyNn':
        comf_ren = input('For next work choose <y> or <n> simbols): ')

    if comf_ren in 'Yy':
        output_filename = 'result' + str(datetime.now().isoformat(timespec='minutes')).replace(':', '_')
    else:
        output_filename = input('Input result file name: ')
    changed_df.to_excel(f'./{output_filename}.xlsx', encoding='cp1251', index = False)

    end_time = time.time()
    print(end_time - start_time)

# Function for changing code 
def changing_code(df):
    date_expected, date_release = df['Код KKS документа'], df['Код KKS документа_new']
    if  not pd.isna(date_expected) and not pd.isna(date_release):
        return f'Смена кода с <{date_expected}> на <{date_release}>'
    else:
        return date_expected

# Function for changing name
def changing_name(df):
    date_expected, date_release = df['Наименование объекта/комплекта РД'], df['Наименование объекта/комплекта РД_new']
    if  not pd.isna(date_expected) and not pd.isna(date_release) and date_expected != date_release:
        return date_release
    else:
        return date_expected

# Function for converting date
def convert_date(row):
    if row in ['в производстве', 'В производстве', None]:
        return row
    else:
        if dayfirst is True:
            return pd.to_datetime(row, dayfirst=dayfirst).date()
        else: 
            return pd.to_datetime(row, format='%Y/%m/%d').date()

# Function for changing_developer
def change_developer(df):
    if ~pd.isna(df['Разработчики РД (актуальные)']):
        return df['Разработчик РД']
    else:
        return df['Разработчики РД (актуальные)']

# Function for changing status
def change_status(df):
    if ~pd.isna(df['Статус текущей ревизии_new']):
        return df['Статус РД в 1С']
    else:
        return df['Статус текущей ревизии_new']

#Function for changing data
def changing_data(row):
    if (row[col] == row[f'{col}_new']) or (pd.isna(row[col]) and pd.isna(row[f'{col}_new'])):
        return None
    else:
        return f'Смена {col.lower()} с <{row[col]}> на <{row[f"{col}_new"]}>'

# read Excel files with current and new data
print('Read excel files with current and new data')
if '.xlsb' in filename_comp:
    with pyxlsb.open_workbook(filename_comp) as wb:
        with wb.get_sheet(1) as sheet:
            data = []
            for row in sheet.rows():
                data.append([item.v for item in row])
    base_df = pd.DataFrame(data[1:], columns=data[0])
    flag = True
else: 
    base_df = pd.read_excel(filename_comp)
new_df.columns = base_columns

merged_df = pd.merge(new_df, base_df, how='left', on='Коды работ по выпуску РД', indicator=True)
missing_columns = merged_df.loc[merged_df['_merge'] == 'left_only']

Read excel files with current and new data


In [72]:
new_df[new_df['Коды работ по выпуску РД'] == '11UBN.E.B00.D-1120']

,Система,Наименование объекта/комплекта РД,Коды работ по выпуску РД,Тип,Пакет РД,Код KKS документа,Статус Заказчика,Текущая ревизия,Статус текущей ревизии,Дата выпуска РД по договору подрядчика,Дата выпуска РД по графику с Заказчиком,Дата статуса Заказчика,Ожидаемая дата выдачи РД в производство,Письма,Источник информации,Разработчики РД (актуальные),Объект,WBS,КС,Примечания
1196,СКУ ЭЧ ЭБ,Компоновочный чертеж электрооборудования,11UBN.E.B00.D-1120,Комплектация/спецификация,02.23.1-U1-3-0029,AKU.0120.11UBN.0.EK.TB0002,ВК - (rev.C01),C01,Comments from T2 to sub-supplier / Замечания о...,None,None,12.05.2022,20.11.2022,исх.№45-03-3707 от 12.05.2022,None,Титан-2=>АЭП,11UBN,E,None,None
